### Notebook stuck?
Note that because of Jupyter and Plotly issues, sometimes the code may stuck at visualization. We recommend **restart the kernels** and try again to see if the issue is resolved.

In [ ]:
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from pathlib import Path
from PIL import Image
from scipy.spatial.transform import Rotation as R

DATASET_ROOT = "/path/to/dataset"  # Specify your own dataset path
SCENE_ID = 0  # Select a scene id

dataset_path = Path(DATASET_ROOT)
print("Chosen ASE data path: ", dataset_path)
print(f"Using Scene {SCENE_ID} for these examples")

scene_path = dataset_path / str(SCENE_ID)

# Part 1: Reading Scene Annotations and 3D Plotting

In [ ]:
from code_snippets.readers import read_points_file, read_trajectory_file, read_language_file

# Load scene point cloud using read_points_file()
points_path = scene_path / "semidense_points.csv.gz"
points = read_points_file(points_path)

# Load a trajectory using read_trajectory_file() 
trajectory_path = scene_path / "trajectory.csv"
trajectory = read_trajectory_file(trajectory_path)

# Load a scene command language using read_language_file()
language_path = scene_path / "ase_scene_language.txt"
entities = read_language_file(language_path)

In [ ]:
from code_snippets.interpreter import language_to_bboxes

# Interpret scene commands into 3D Boxes
entity_boxes = language_to_bboxes(entities)

In [ ]:
# Plot each modality together in 3D
from code_snippets.plotters import plot_point_cloud, plot_trajectory, plot_box_wireframe

plot_traces = []
# Create a trace for the pointcloud using plot_point_cloud()
plot_traces.append(plot_point_cloud(points))
# Create a trace for the trajectory using plot_trajectory()
plot_traces.append(plot_trajectory(trajectory))

# Create a trace for each entity box in the form of a wireframe using plot_box_wireframe()
for entity_box in entity_boxes:
    plot_traces.append(plot_box_wireframe(entity_box))

fig = go.Figure(data=plot_traces)
fig.update_layout(
    template="plotly_dark",
    scene={
        "xaxis": {"showticklabels": False, "title": ""},
        "yaxis": {"showticklabels": False, "title": ""},
        "zaxis": {"showticklabels": False, "title": ""},
    },
)
fig.show()

In [ ]:
# Alternatively, we include a plotting function that will handle all of the steps for you.
from code_snippets.plotters import plot_3d_scene

plot_3d_scene(
    language_path=language_path,
    points_path=points_path,
    trajectory_path=trajectory_path
)

# Part 2: Loading and Plotting Images and Image Annotations

In [ ]:
def random_bright_colormap(num_colors=1024):
    bright_colors = np.random.rand(num_colors, 3)
    bright_colors = (bright_colors + 1) / 2
    return colors.ListedColormap([c for c in bright_colors])

scene_path = dataset_path / str(SCENE_ID)

rgb_dir = scene_path / "rgb"
depth_dir = scene_path / "depth"
instance_dir = scene_path / "instances"

# Choose a random frame to plot from the scene's images
num_frames = len(list(rgb_dir.glob("*.jpg")))
frame_idx = np.random.randint(num_frames)

# Load images
frame_id = str(frame_idx).zfill(7)

rgb_path = rgb_dir / f"vignette{frame_id}.jpg"
depth_path = depth_dir / f"depth{frame_id}.png"
instance_path = instance_dir / f"instance{frame_id}.png"

rgb = Image.open(rgb_path)
depth = Image.open(depth_path)
instances = Image.open(instance_path)

# Note: Images are rotated to upright for visualization.
# However, the camera calibration is for the original orientation.
rgb_to_plot = rgb.rotate(-90, expand=True)
depth_to_plot = depth.rotate(-90, expand=True)
instances_to_plot = instances.rotate(-90, expand=True)

# Plot the images
fig, axes = plt.subplots(ncols=3, nrows=1, figsize=(15, 5), dpi=300)
axes[0].imshow(rgb_to_plot)
axes[0].set_title("RGB Image")
axes[1].imshow(np.array(depth_to_plot), cmap="plasma")
axes[1].set_title("Metric Depth (mm)")
axes[2].imshow(np.array(instances_to_plot), cmap=random_bright_colormap())
axes[2].set_title("Instance Map")
for ax in axes:
    ax.axis("off")
plt.show()

# Part 3: Project Points into Images

In [ ]:
from projectaria_tools.projects import ase

def transform_3d_points(transform, points):
    N = len(points)
    points_h = np.concatenate([points, np.ones((N, 1))], axis=1)
    transformed_points_h = (transform @ points_h.T).T
    transformed_points = transformed_points_h[:, :-1]
    return transformed_points
    
# Load camera calibration
device = ase.get_ase_rgb_calibration()

# Load the trajectory using read_trajectory_file() 
trajectory_path = scene_path / "trajectory.csv"
trajectory = read_trajectory_file(trajectory_path)

# Load scene point cloud using read_points_file()
points_path = scene_path / "semidense_points.csv.gz"
points_world = read_points_file(points_path)

# Choose and load random frame to plot from the scene's images
num_frames = len(list(rgb_dir.glob("*.jpg")))
frame_idx = np.random.randint(num_frames)
frame_id = str(frame_idx).zfill(7)
rgb_path = rgb_dir / f"vignette{frame_id}.jpg"
rgb = Image.open(rgb_path)

# Transform the points into the device coordinate frame
T_world_from_device = trajectory["Ts_world_from_device"][frame_idx]
T_device_from_world = np.linalg.inv(T_world_from_device)
points_device = transform_3d_points(T_device_from_world, points_world)

# Transform the points into the RGB camera coordinate frame
T_device_from_camera = device.get_transform_device_camera().matrix()
T_camera_from_device = np.linalg.inv(T_device_from_camera)
points_device = transform_3d_points(T_camera_from_device, points_device)

# Project points into the image
points_image = []
depths = []
for point_device in points_device:
    point_image = device.project(point_device)
    if point_image is not None:
        points_image.append(point_image)
points_image = np.stack(points_image)

# Overlay projected points onto image
plt.imshow(rgb)
plt.scatter(points_image[:, 0], points_image[:, 1], s=0.01, alpha=0.3, c="#FFFFFF")
plt.axis("off")
plt.show()